In [1]:
# Loading of libraries
import numpy as np
import scipy.spatial.distance
import matplotlib.pyplot as plt
from numba import jit, autojit
import time
import warnings
import circles

%matplotlib inline

from ipyparallel import Client

In [2]:
# Design variables
AverageRadius            = 1
DeviationRadius          = AverageRadius / 10
ParticleCount            = 400
NeighbourDistance        = 2.7 * AverageRadius
SpringConstant           = 1
Chi                      = 1         # Rotational viscosity    - 1 in natural units
Zeta                     = 1         # Translational visocsity - 1 in natural units
k                        = 1         # Sprint constant         - 1 in natural units

# Simulation variables
NeighbourCutoff          = 2.7
TimeStep                 = 0.05
L_align                  = 0.2       # Range from 0.1 to 1
L_noise                  = 0.03      
L_F_in                   = 0.3
L_T_in                   = 3
L_self                   = 0.07      #  Range from 0.04 to 0.08
T_align                  = L_align * k * Chi / Zeta
T_noise                  = np.sqrt(L_noise * 2 * k * Chi**2 / (TimeStep * Zeta))
F_in                     = L_F_in * k * AverageRadius
T_in                     = L_T_in * k * Chi / Zeta
F_self                   = L_self * k * AverageRadius

# Initiate the pengiuins
InitialSpacing           = 2*AverageRadius
Radius                   = np.random.normal(AverageRadius, DeviationRadius, ParticleCount)
Location                 = np.array(np.vstack((
                             np.tile(np.arange(0,10), ParticleCount / 10) * InitialSpacing,
                             np.repeat(np.arange(0,ParticleCount / 10), 10) * InitialSpacing
                           )), dtype=np.float64).T + np.random.normal(0,AverageRadius/10,(ParticleCount,2))
Velocity                 = np.zeros((ParticleCount, 2))
Orientation              = np.ones(ParticleCount) * np.pi / 2 + np.random.uniform(-np.pi/4, np.pi/4, ParticleCount)
AngularVelocity          = np.zeros(ParticleCount)